In [3]:
# Install required libraries
!pip install pandas openpyxl transformers

In [4]:
# Import necessary libraries
import pandas as pd

# Load the Excel file and display sheet names
file_path = 'Miquela Instagram Post_clean data.xlsx'  # Adjust path if necessary
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names  # Get the sheet names
print("Available sheets:", sheet_names)

Available sheets: ['Miquela Post', 'Netflix', 'Burberry', 'Orly', 'Spotify', 'Samsung&Seventeen', 'Spotify2', 'Samsung&Sev.2', 'Pot LA', 'KitstoKick', 'Burberry2', 'Elle Mexico', 'Off White', 'Officiel Singapore', 'CalvinKlein', 'CalvinKlein2', 'YouTube Coachella', 'LACMA', 'PlannedParenthood', 'Tinder', 'Ugg', 'AWang', 'Vogue', 'AppleGoogle', 'HighSnobiety', 'VMagazine', 'Prada', 'Monclear', 'PaperMag', 'RichardQ', 'CampFlogGnaw', 'SamsungGalaxy', 'YouTube&Spotify', 'Kimshiustudio', 'BigNoise', 'AMAS', 'InnOut Burger', 'VMAS', 'Comic Con', 'Bloom&PlumeCoffee', 'SamsungGalaxy2', 'NotionMag', 'YouTube&Coachella', 'ES Mag', 'BorderAngels', 'Brud', 'RichardQuin2', 'Native Instruments', 'DazedBeauty', 'Club404notfound', 'SamsungGalaxy3']


In [5]:
# Filter out the "Miquela Post" and "Netflix" sheets
exclude_sheets = ["Miquela Post", "Netflix"]
sheet_names = [sheet for sheet in xls.sheet_names if sheet not in exclude_sheets]
print("Processing sheets:", sheet_names)


Processing sheets: ['Burberry', 'Orly', 'Spotify', 'Samsung&Seventeen', 'Spotify2', 'Samsung&Sev.2', 'Pot LA', 'KitstoKick', 'Burberry2', 'Elle Mexico', 'Off White', 'Officiel Singapore', 'CalvinKlein', 'CalvinKlein2', 'YouTube Coachella', 'LACMA', 'PlannedParenthood', 'Tinder', 'Ugg', 'AWang', 'Vogue', 'AppleGoogle', 'HighSnobiety', 'VMagazine', 'Prada', 'Monclear', 'PaperMag', 'RichardQ', 'CampFlogGnaw', 'SamsungGalaxy', 'YouTube&Spotify', 'Kimshiustudio', 'BigNoise', 'AMAS', 'InnOut Burger', 'VMAS', 'Comic Con', 'Bloom&PlumeCoffee', 'SamsungGalaxy2', 'NotionMag', 'YouTube&Coachella', 'ES Mag', 'BorderAngels', 'Brud', 'RichardQuin2', 'Native Instruments', 'DazedBeauty', 'Club404notfound', 'SamsungGalaxy3']


In [6]:
# Function to load data from a specific sheet
def load_sheet(sheet_name):
    """
    Loads data from a given sheet and returns the comments and caption.
    """
    df = pd.read_excel(xls, sheet_name=sheet_name, header=None)
    caption = df.iloc[1, 0]  # Second row (index 1) is the caption
    comments = df.iloc[2:, 0].dropna().tolist()  # All rows from third onwards are comments
    return caption, comments

# Load data from all sheets (excluding specified ones) into a dictionary
data = {sheet: load_sheet(sheet) for sheet in sheet_names}

In [7]:
# Prepare prompts for topic modeling
def create_prompt(caption, comments):
    """
    Creates a prompt for topic modeling from a post's caption and comments.
    """
    comment_snippet = " ".join(comments[:20])  # Use first 20 comments for brevity
    prompt = (
        f"The following post has the caption: '{caption}'. "
        f"Here are some comments on the post: {comment_snippet}. "
        f"Please identify 5-6 major themes or topics discussed in this content. "
        "Additionally, provide quantitative dimensions for each topic based on their prevalence in the comments."
    )
    return prompt

# Generate prompts for all sheets
prompts = {sheet: create_prompt(*data[sheet]) for sheet in sheet_names}


In [8]:
# Import libraries for transformers
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load Flan-T model and tokenizer
model_name = "google/flan-t5-large"  # You can adjust this based on available compute resources
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Create a pipeline for text generation
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [10]:
# Import libraries for transformers
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Load Flan-T model and tokenizer
model_name = "google/flan-t5-large"  # You can adjust this based on available compute resources
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Specify the device for the pipeline
# device=0 uses the first GPU if available; device=-1 uses the CPU
import torch
device = 0 if torch.cuda.is_available() else -1

# Create a pipeline for text generation with device support
flan_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=device)

# Confirm the device being used
print(f"Flan-T pipeline loaded on {'GPU' if device >= 0 else 'CPU'} (device={device}).")


Flan-T pipeline loaded on GPU (device=0).


In [11]:
# Define a function to split sheets into batches
def batch_sheets(sheets, batch_size):
    """
    Splits a list of sheet names into smaller batches.
    """
    for i in range(0, len(sheets), batch_size):
        yield sheets[i:i + batch_size]

# Set batch size and create batches
batch_size = 5  # Adjust as needed
sheet_batches = list(batch_sheets(sheet_names, batch_size))
print(f"Number of batches: {len(sheet_batches)}")


Number of batches: 10


In [12]:
# Run topic modeling for each sheet
def get_topics(prompt):
    """
    Uses the Flan-T LLM to extract topics and dimensions from the given prompt.
    """
    result = flan_pipeline(prompt, max_length=512, num_return_sequences=1)[0]['generated_text']
    return result

# Apply the model to all prompts
results = {sheet: get_topics(prompt) for sheet, prompt in prompts.items()}


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (528 > 512). Running this sequence through the model will result in indexing errors


In [13]:
# Initialize a list to store all results
all_results = []

# Loop through each batch of sheets
for batch_idx, batch in enumerate(sheet_batches):
    print(f"Processing batch {batch_idx + 1}/{len(sheet_batches)}: {batch}")

    # Load data for the current batch
    batch_data = {sheet: load_sheet(sheet) for sheet in batch}

    # Generate prompts for the current batch
    batch_prompts = {sheet: create_prompt(*batch_data[sheet]) for sheet in batch}

    # Run topic modeling for the current batch
    batch_results = {sheet: get_topics(prompt) for sheet, prompt in batch_prompts.items()}

    # Append batch results to the global results
    all_results.extend(batch_results.items())

    # Save intermediate results after processing each batch
    intermediate_df = pd.DataFrame(all_results, columns=['Sheet', 'Topics'])
    intermediate_df.to_csv(f'topic_modeling_results_batch_{batch_idx + 1}.csv', index=False)
    print(f"Batch {batch_idx + 1} results saved.")


Processing batch 1/10: ['Burberry', 'Orly', 'Spotify', 'Samsung&Seventeen', 'Spotify2']
Batch 1 results saved.
Processing batch 2/10: ['Samsung&Sev.2', 'Pot LA', 'KitstoKick', 'Burberry2', 'Elle Mexico']
Batch 2 results saved.
Processing batch 3/10: ['Off White', 'Officiel Singapore', 'CalvinKlein', 'CalvinKlein2', 'YouTube Coachella']
Batch 3 results saved.
Processing batch 4/10: ['LACMA', 'PlannedParenthood', 'Tinder', 'Ugg', 'AWang']
Batch 4 results saved.
Processing batch 5/10: ['Vogue', 'AppleGoogle', 'HighSnobiety', 'VMagazine', 'Prada']
Batch 5 results saved.
Processing batch 6/10: ['Monclear', 'PaperMag', 'RichardQ', 'CampFlogGnaw', 'SamsungGalaxy']
Batch 6 results saved.
Processing batch 7/10: ['YouTube&Spotify', 'Kimshiustudio', 'BigNoise', 'AMAS', 'InnOut Burger']
Batch 7 results saved.
Processing batch 8/10: ['VMAS', 'Comic Con', 'Bloom&PlumeCoffee', 'SamsungGalaxy2', 'NotionMag']
Batch 8 results saved.
Processing batch 9/10: ['YouTube&Coachella', 'ES Mag', 'BorderAngels', 